In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

runcycleid = etl_helpers.start_run_cycle("factMODRequestDocumentDeleted")

os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    today = str(datetime.date.today())

    df_existing = spark.sql("SELECT max(created_at) as createddate FROM factmodrequestdocumentdeleted")
    maxcreatedate = df_existing.first().createddate
    maxcreatedate_str = maxcreatedate.strftime("%Y-%m-%d %H:%M:%S")

    query = f"""
        SELECT
        ministryrequestid,
        documentdeletedid,
        created_at,
        createdby,
        filepath
        from docreviewer.documentdeleted
        where created_at > '{maxcreatedate_str}'
        """

    print(query)

    df = spark.sql(query)
    df.show()


    # order of columns here is important!
    df_mapped = df.selectExpr(
            "documentdeletedid AS documentdeletedid",
            "filepath AS filepath",
            "createdby AS createdby",
            "created_at AS created_at",
            "ministryrequestid AS foiministryrequestid",
            f"{runcycleid} as runcycleid",
            
        )
    df_mapped.show()
    etl_helpers.append_with_incrementing_id(df_mapped, table_name="factmodrequestdocumentdeleted", id_column="id", database="hive_metastore.default")
    etl_helpers.end_run_cycle(runcycleid, 't', "factMODRequestDocumentDeleted")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factMODRequestDocumentDeleted", "Credentials not available")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factMODRequestDocumentDeleted")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factMODRequestDocumentDeleted", f"An error occurred: {e}")